# Working with Time-Series Data in a Consistent Bayesian Framework
---

Copyright 2017 Michael Pilosov


### Import Libraries
_(should be 2.7 and 3.x compatible) _

In [1]:
# Mathematics and Plotting
from HelperFuns import * # pyplot wrapper functions useful for visualizations, numpy, scipy, etc.
%matplotlib inline
plt.rcParams.update({'font.size': 14})
plt.rcParams['figure.figsize'] = 5, 5

# Interactivity
from ipywidgets import *

---
## Defining the Parameter to Observables (PtO) Map 

---
Consider the Ordinary Differential Equation Initival Value Problem given by  

$$
\partial_t u(t) = -u(t) \\
u(0) = \lambda_0
$$

The solution to this problem is $u(t) = \lambda_0 \,e^{-t}$.

Suppose $\lambda_0$ is some uncertain input parameter that we are trying to estimate through experimental observations. 

Suppose we know that $\lambda_0 \in [0, 2]$ with uniform probability and that we hope to infer the parameter by observing the system $u$ at K evenly spaced intervals in the interval $[T_0, T]$, where $T > T_0 > 0$. 

In order to define our Parameter-to-Observables Map, we want to transform these observations into a single measurement, which we do by looking at the **Mean Squared Error (2-norm) between the observations and the model predictions**:  

$$
O(\lambda) = \frac{1}{K} \sum_{k=1}^{K} \frac{(\lambda e^{-k\Delta_t} - \lambda_0e^{-k\Delta_t}) ^ 2}{\sigma_k^2} = \frac{1}{K}  \sum_{k=1}^{K} \frac{( (\lambda- \lambda_0) e^{-k\Delta_t} ) ^ 2 }{ \sigma_k^2}
$$
$$
= \frac{(\lambda- \lambda_0)^2}{K}  \sum_{k=1}^{K}  \left ( \frac{e^{-k\Delta_t}}{\sigma_k} \right )^2, \quad \text{where } \Delta_t\equiv T_0 + \frac{T - T_0}{K}
$$

Observe that the division of each measurement by a standard deviation is consistent with the formulation of an additive error statistical model familiar to the usual Bayesian formulation:  

$$
y_i = \beta\, x_i + \epsilon_i, \quad \epsilon_i \sim N(0,\sigma_i)
$$

For a step-by-step walkthrough, please see the CBayes_TS.ipynb file.
Below you will find an all-in-one version. 


### Define some functions for the sandbox

In [2]:
def sandbox(num_samples = int(1E4), lam_bound = [0,2], lam0=0.5, num_observations=1,
            T=[0.1,1], uncertainty = 0.05, sd = 1):
    # NOTE this version only uses constant variances for the sake
    # of interactivity. 
    sigma = sd*np.ones(num_observations)
    T_start, T_end = T
    if T_end < T_start:
        print('Error: end time is before start time. Switching them now.')
        T_temp = T_end
        T_end = T_start
        T_start = T_temp
    if num_observations == 1:
        print('K=1 specified, This is a single observation at t = %f.'%T_start)
    t = np.linspace(T_start, T_end, num_observations)
    def PtO_fun(lam):
        return ((lam - lam0)**2/num_observations)*np.sum( [ np.power(
            [ np.exp(-t[k])/sigma[k] ], 2)[0] 
            for k in range(int(num_observations))          ], 0 )
    
    # Sample the Parameter Space
    a, b = lam_bound
    lam = np.random.uniform(a, b, size = (1, int(num_samples)) ) # standard uniform
    # Map to Data Space
    D = PtO_fun(lam)
    print('dimensions :  lambda = ' + str(lam.shape) + '   D = ' + str(D.shape) )
    # Perform KDE to estimate the pushforward
    pf_dens = gkde(D) # compute KDE estimate of it
    # Specify Observed Measure - Uniform Density
    
    obs_dens = sstats.uniform(0,uncertainty) # 1D only
    # Solve the problem
    r = obs_dens.pdf( D ) / pf_dens.evaluate(D) # vector of ratios evaluated at all the O(lambda)'s
    M = np.max(r)
    eta_r = r[0]/M
    
    res = 50;
    max_x = 1.1;
    # Plot stuff
    plt.rcParams['figure.figsize'] = (18, 6)
    plt.figure()
    plt.subplot(1, 3, 1)
    x = np.linspace(0, max_x, res)
    plt.plot(x,pf_dens.evaluate(x))
    plt.title('Pushforward of Prior')
    plt.xlabel('O(lambda)')
    
    plt.subplot(1, 3, 2)
    xx = np.linspace(0, max_x, res)
    plt.plot(xx,obs_dens.pdf(xx))
    plt.title('Observed Density')
    plt.xlabel('O(lambda)')

    plt.subplot(1, 3, 3)
    plt.scatter(lam,eta_r)
    # plt.plot(lam_accept, gkde(lam_accept))
    plt.scatter(lam0, 0.05)
    # plt.title('Posterior Distribution\nof Uniform Observed Density \nwith bound = %1.2e'%uni_max)
    plt.xlabel('Lambda')
    # # OPTIONAL:
    # pr = 0.2 # percentage view-window around true parameter.
#     plt.xlim(lam0*np.array([1-pr,1+pr]))
    plt.xlim([a,b])
    plt.show()



Jump to [Extra Visualizations](#Extra-Visualizations) at the bottom of the worksheet to see the prior compared to the posterior as well as the pushforward of the posterior compared to the observed density. 

---

# All-in-One Sandbox!
_Run the cells below to start experimenting_

In [27]:
interact_manual(sandbox, 
        num_samples = IntSlider(value=500, 
            min=int(5E2), max=int(5E4), step=500, description='Samp. $N$ ='), 
        lam_bound = FloatRangeSlider(value=[0.0, 2.0], 
            min=-5.0, max = 5.0, step=0.25, description='Param $\Lambda \in$'),
        lam0 = FloatSlider(value=1.0, 
            min=0.25, max=1.75, step=0.05, description='IC: $\lambda_0$ ='), 
        num_observations = IntSlider(value=10, 
            min=1, max=50, description='Obs. $K$ ='), 
        T = FloatRangeSlider( value=[0.5, 1], min=0.1, max=7.5, step=0.1,
            description='$t\in [T_0, T]$:', orientation='horizontal',
            readout=True, readout_format='.1f'), 
        uncertainty = FloatSlider(value=0.1, 
            min=0.005, max=0.5, step=0.005, description='Invert MSE $\leq$'),
        sd = FloatSlider(value=1, 
            min=0.15, max=1.85, step=0.05, description='Constant $\sigma$:') )
plt.show()

A Jupyter Widget